<a href="https://colab.research.google.com/github/agemagician/CodeTrans/blob/main/prediction/single%20task/api%20generation/small_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<h3>Generate the api based on the description using codeBert single task training model</h3>**
<h4>You can make free prediction online through this 
<a href="https://huggingface.co/SEBIS/code_trans_t5_small_api_generation">Link</a></h4> (When using the prediction online, you need to parse and tokenize the code first.)

**1. Load necessry libraries including huggingface transformers**

In [ ]:
!pip install -q transformers

     |████████████████████████████████| 1.3MB 5.6MB/s 
     |████████████████████████████████| 890kB 16.9MB/s 
     |████████████████████████████████| 2.9MB 18.8MB/s 
     |████████████████████████████████| 1.1MB 40.0MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline

**2. Load the token classification pipeline and load it into the GPU if avilabile**

In [ ]:
pipeline = SummarizationPipeline(
    model=AutoModelWithLMHead.from_pretrained("SEBIS/code_trans_t5_small_api_generation"),
    tokenizer=AutoTokenizer.from_pretrained("SEBIS/code_trans_t5_small_api_generation", skip_special_tokens=True),
    device=0
)

/usr/local/lib/python3.6/dist-packages/transformers/modeling_auto.py:837: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


Some weights of T5ForConditionalGeneration were not initialized from the model checkpoint at SEBIS/code_trans_t5_small_api_generation and are newly initialized: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**3 Give the description for generating api, parse and tokenize it**

In [ ]:
description = "parse the uses licence node of this package, if any, and returns the license definition if theres" #@param {type:"raw"}

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

def englishTokenizer(sentence):
    result = []
    tokens = word_tokenize(sentence)
    for t in tokens:
        if( not len(t)>50):
            result.append(t)
    return ' '.join(result)

tokenized_description = englishTokenizer(description)
print("tokenized description: " + tokenized_description)

tokenized description: parse the uses licence node of this package , if any , and returns the license definition if theres


**4. Make Prediction**

In [ ]:
pipeline([tokenized_description])

Your max_length is set to 512, but you input_length is only 25. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=50)


[{'summary_text': 'Node . getNodeValue Map < String , String > . get'}]